<a href="https://colab.research.google.com/github/jimwhite/babyagi-langchain/blob/main/baby_agi_with_programming_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BabyAGI with Tools including Coding Agent

This is a copy of LangChain's [Baby AGI with Tools](https://python.langchain.com/en/latest/use_cases/agents/baby_agi_with_agent.html) with a few tweaks so it should just run in [Google Colab](https://colab.research.google.com/).  You could try "Run all" but probably better to do one block at a time.  There are prompts for entering your OpenAI and Search tool (either [Serp API](https://python.langchain.com/en/latest/ecosystem/serpapi.html) or [Google Custom Search API](https://python.langchain.com/en/latest/ecosystem/google_search.html)) keys.

**N.B.** This version includes the LangChain Python REPL tool and so this BabyAGI can write and execute arbitrary code in this process.  There is no isolation/containerization (yet) so **USE AT YOUR OWN RISK!**

License is [MIT License](https://raw.githubusercontent.com/hwchase17/langchain/master/LICENSE).

## Install and Import Required Modules

In [1]:
%pip install -U langchain openai tiktoken > /dev/null
%pip install faiss-cpu google-search-results > /dev/null

In [2]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from getpass import getpass

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores.base import VectorStore
from langchain.vectorstores import FAISS
from langchain.chains.base import Chain
from pydantic import BaseModel, Field, ValidationError

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [ ]:
# Prompt for manual entry of OpenAI API key if it isn't already defined.
if not 'OPENAI_API_KEY' in os.environ:
  print("Please enter your OpenAI API key:")
  os.environ['OPENAI_API_KEY'] = getpass()

In [4]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [5]:
class TaskCreationChain(LLMChain):
    """Chain to generates tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_creation_template = (
            "You are a task creation AI that uses the result of an execution agent"
            " to create new tasks with the following objective: {objective},"
            " The last completed task has the result: {result}."
            " This result was based on this task description: {task_description}."
            " These are incomplete tasks: {incomplete_tasks}."
            " Based on the result, create new tasks to be completed"
            " by the AI system that do not overlap with incomplete tasks."
            " Return the tasks as an array."
        )
        prompt = PromptTemplate(
            template=task_creation_template,
            input_variables=[
                "result",
                "task_description",
                "incomplete_tasks",
                "objective",
            ],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [6]:
class TaskPrioritizationChain(LLMChain):
    """Chain to prioritize tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_prioritization_template = (
            "You are a task prioritization AI tasked with cleaning the formatting of and reprioritizing"
            " the following tasks: {task_names}."
            " Consider the ultimate objective of your team: {objective}."
            " Do not remove any tasks. Return the result as a numbered list, like:"
            " #. First task"
            " #. Second task"
            " Start the task list with number {next_task_id}."
        )
        prompt = PromptTemplate(
            template=task_prioritization_template,
            input_variables=["task_names", "next_task_id", "objective"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

Only one of these Search APIs will be used so just run the one you want. 

---



In [ ]:
from langchain import SerpAPIWrapper

# Get Serp API keys here: https://serpapi.com/

try:
  search
except NameError:
  # Prompt for manual entry of Serp API key if it isn't already defined.
  if not os.environ.get('SERPAPI_API_KEY', None):
    print("Please enter your Serp API key (or press Enter to skip):")
    os.environ['SERPAPI_API_KEY'] = getpass()
  try:
    if os.environ.get('SERPAPI_API_KEY', None):
      search = SerpAPIWrapper()
  except ValidationError as ex:
    os.environ['SERPAPI_API_KEY'] = None
    print(ex)

In [ ]:
from langchain import GoogleSearchAPIWrapper

# Set up Google Custom Search API here: https://programmablesearchengine.google.com/
# Docs here: https://python.langchain.com/en/latest/ecosystem/google_search.html

try:
  search
except NameError:
  # Prompt for manual entry of Google Custom Search ID and API key if it isn't already defined.
  if not os.environ.get('GOOGLE_CSE_ID', None):
    os.environ['GOOGLE_CSE_ID'] = input("Please enter your Google Custom Search ID (or press Enter to skip):")
  if os.environ.get('GOOGLE_CSE_ID', None) and not os.environ.get('GOOGLE_API_KEY', None):
    print("Please enter your Google Search API key:")
    os.environ['GOOGLE_API_KEY'] = getpass()
  try:
    if os.environ.get('GOOGLE_CSE_ID', None) and os.environ.get('GOOGLE_API_KEY', None):
      search = GoogleSearchAPIWrapper()
  except ValidationError as ex:
    os.environ['GOOGLE_CSE_ID'] = None
    os.environ['GOOGLE_API_KEY'] = None
    print(ex) 

In [8]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.utilities import PythonREPL

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)

python_repl = PythonREPL()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
    Tool(
        name="Python REPL",
        func=python_repl.run,
        description="useful for running Python code. Input: Python code text. Output: the output from the print function calls.",
    )
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

### Define the BabyAGI Controller

BabyAGI composes the chains defined above in a (potentially-)infinite loop.

In [9]:
def get_next_task(
    task_creation_chain: LLMChain,
    result: Dict,
    task_description: str,
    task_list: List[str],
    objective: str,
) -> List[Dict]:
    """Get the next task."""
    incomplete_tasks = ", ".join(task_list)
    response = task_creation_chain.run(
        result=result,
        task_description=task_description,
        incomplete_tasks=incomplete_tasks,
        objective=objective,
    )
    new_tasks = response.split("\n")
    return [{"task_name": task_name} for task_name in new_tasks if task_name.strip()]

In [10]:
def prioritize_tasks(
    task_prioritization_chain: LLMChain,
    this_task_id: int,
    task_list: List[Dict],
    objective: str,
) -> List[Dict]:
    """Prioritize tasks."""
    task_names = [t["task_name"] for t in task_list]
    next_task_id = int(this_task_id) + 1
    response = task_prioritization_chain.run(
        task_names=task_names, next_task_id=next_task_id, objective=objective
    )
    new_tasks = response.split("\n")
    prioritized_task_list = []
    for task_string in new_tasks:
        if not task_string.strip():
            continue
        task_parts = task_string.strip().split(".", 1)
        if len(task_parts) == 2:
            task_id = task_parts[0].strip()
            task_name = task_parts[1].strip()
            prioritized_task_list.append({"task_id": task_id, "task_name": task_name})
    return prioritized_task_list

In [17]:
def _get_top_tasks(vectorstore, query: str, k: int) -> List[str]:
    """Get the top k tasks based on the query."""
    results = vectorstore.similarity_search_with_score(query, k=k)
    if not results:
        return []
    sorted_results, _ = zip(*sorted(results, key=lambda x: x[1], reverse=True))
    return [str(item.metadata["task"]) for item in sorted_results]


def execute_task(
    vectorstore, execution_chain: LLMChain, objective: str, task: str, k: int = 5
) -> str:
    """Execute a task."""
    context = _get_top_tasks(vectorstore, query=objective, k=k)
    return execution_chain.run(objective=objective, context=context, task=task)

In [18]:
class BabyAGI(Chain, BaseModel):
    """Controller model for the BabyAGI agent."""

    task_list: deque = Field(default_factory=deque)
    task_creation_chain: TaskCreationChain = Field(...)
    task_prioritization_chain: TaskPrioritizationChain = Field(...)
    execution_chain: AgentExecutor = Field(...)
    task_id_counter: int = Field(1)
    vectorstore: VectorStore = Field(init=False)
    max_iterations: Optional[int] = None

    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    def add_task(self, task: Dict):
        self.task_list.append(task)

    def print_task_list(self):
        print("\033[95m\033[1m" + "\n*****TASK LIST*****\n" + "\033[0m\033[0m")
        for t in self.task_list:
            print(str(t["task_id"]) + ": " + t["task_name"])

    def print_next_task(self, task: Dict):
        print("\033[92m\033[1m" + "\n*****NEXT TASK*****\n" + "\033[0m\033[0m")
        print(str(task["task_id"]) + ": " + task["task_name"])

    def print_task_result(self, result: str):
        print("\033[93m\033[1m" + "\n*****TASK RESULT*****\n" + "\033[0m\033[0m")
        print(result)

    @property
    def input_keys(self) -> List[str]:
        return ["objective"]

    @property
    def output_keys(self) -> List[str]:
        return []

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Run the agent."""
        objective = inputs["objective"]
        first_task = inputs.get("first_task", "Make a todo list")
        self.add_task({"task_id": 1, "task_name": first_task})
        num_iters = 0
        while True:
            if self.task_list:
                self.print_task_list()

                # Step 1: Pull the first task
                task = self.task_list.popleft()
                self.print_next_task(task)

                # Step 2: Execute the task
                result = execute_task(
                    self.vectorstore, self.execution_chain, objective, task["task_name"]
                )
                this_task_id = int(task["task_id"])
                self.print_task_result(result)

                # Step 3: Store the result in Pinecone
                result_id = f"result_{task['task_id']}"
                self.vectorstore.add_texts(
                    texts=[result],
                    metadatas=[{"task": task["task_name"]}],
                    ids=[result_id],
                )

                # Step 4: Create new tasks and reprioritize task list
                new_tasks = get_next_task(
                    self.task_creation_chain,
                    result,
                    task["task_name"],
                    [t["task_name"] for t in self.task_list],
                    objective,
                )
                for new_task in new_tasks:
                    self.task_id_counter += 1
                    new_task.update({"task_id": self.task_id_counter})
                    self.add_task(new_task)
                self.task_list = deque(
                    prioritize_tasks(
                        self.task_prioritization_chain,
                        this_task_id,
                        list(self.task_list),
                        objective,
                    )
                )
            num_iters += 1
            if self.max_iterations is not None and num_iters == self.max_iterations:
                print(
                    "\033[91m\033[1m" + "\n*****TASK ENDING*****\n" + "\033[0m\033[0m"
                )
                break
        return {}

    @classmethod
    def from_llm(
        cls, llm: BaseLLM, vectorstore: VectorStore, verbose: bool = False, **kwargs
    ) -> "BabyAGI":
        """Initialize the BabyAGI Controller."""
        task_creation_chain = TaskCreationChain.from_llm(llm, verbose=verbose)
        task_prioritization_chain = TaskPrioritizationChain.from_llm(
            llm, verbose=verbose
        )
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        tool_names = [tool.name for tool in tools]
        agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
        agent_executor = AgentExecutor.from_agent_and_tools(
            agent=agent, tools=tools, verbose=True
        )
        return cls(
            task_creation_chain=task_creation_chain,
            task_prioritization_chain=task_prioritization_chain,
            execution_chain=agent_executor,
            vectorstore=vectorstore,
            **kwargs,
        )

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [19]:
OBJECTIVE = "Show me the first 10 digits of the square root of 3.5 in reverse"

In [20]:
llm = OpenAI(temperature=0)

In [21]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, verbose=verbose, max_iterations=max_iterations
)

In [22]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to create a todo list
Action: TODO
Action Input: Show me the first 10 digits of the square root of 3.5 in reverse
Observation:  order

1. Calculate the square root of 3.5
2. Round the result to 10 decimal places
3. Convert the result to a string
4. Reverse the string
5. Print the first 10 digits of the reversed string
Thought: I now know the final answer
Final Answer: 5.839396887

> Finished chain.

*****TASK RESULT*****

5.839396887

*****TASK LIST*****

2: Find the prime factors of 5.839396887.
3: Create a graph of the relationship between the square root of 3.5 and 5.839396887.
4: Calculate the square root of 3.5 and reverse the first 10 digits.
5: Find the sum of the first 10 digits of the square root of 3.5.
6: Find the average of the first 10 digits of the square root of 3.5.
7: Find the median of the first 10 digits of the square root of 3.5

{'objective': 'Show me the first 10 digits of the square root of 3.5 in reverse'}